In [26]:
import cv2
import numpy as np
import os
import random
import pandas as pd
import re

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import os

folder_path = '/content/drive/MyDrive/dataset'

# Check the contents of the folder
os.listdir(folder_path)

['LPPILOTSITTING2', 'LPPILOTLAT2', 'LPPILOTLAT1', 'AM12']

In [19]:
# Define the root folder in your Google Drive containing subfolders with images
root_folder = '/content/drive/MyDrive/dataset'

In [9]:
#We will now see how many files are there in each folder before augmenting
# Loop through each subfolder in the root folder
for subfolder in os.listdir(root_folder):
    subfolder_path = os.path.join(root_folder, subfolder)

    # Check if the subfolder is a directory
    if os.path.isdir(subfolder_path):
        # Initialize a counter for image files
        image_count = 0

        # Loop through files in the subfolder
        for filename in os.listdir(subfolder_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                image_count += 1

        print(f"Subfolder '{subfolder}' contains {image_count} image(s).")

Subfolder 'LPPILOTSITTING2' contains 373 image(s).
Subfolder 'LPPILOTLAT2' contains 372 image(s).
Subfolder 'LPPILOTLAT1' contains 285 image(s).
Subfolder 'AM12' contains 373 image(s).


In [10]:
# Function to apply random brightness adjustment
def random_brightness(image):
    brightness_factor = random.uniform(0.5, 2.0)
    return cv2.convertScaleAbs(image, alpha=brightness_factor, beta=0)

# Function to apply random contrast adjustment
def random_contrast(image):
    contrast_factor = random.uniform(0.5, 2.0)
    return cv2.convertScaleAbs(image, alpha=contrast_factor, beta=0)

# Function to apply random left-right flip
def random_flip_left_right(image):
    if random.choice([True, False]):
        return cv2.flip(image, 1)
    return image

# Function to apply random up-down flip
def random_flip_up_down(image):
    if random.choice([True, False]):
        return cv2.flip(image, 0)
    return image

# Function to apply random hue adjustment
def random_hue(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hue_shift = random.randint(-10, 10)
    hsv_image[..., 0] = (hsv_image[..., 0] + hue_shift) % 180
    return cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)

# Function to apply random JPEG quality adjustment
def random_jpeg_quality(image):
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), random.randint(50, 100)]
    _, encoded_image = cv2.imencode('.jpg', image, encode_param)
    return cv2.imdecode(encoded_image, 1)

# Function to apply random saturation adjustment
def random_saturation(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    saturation_factor = random.uniform(0.5, 2.0)
    hsv_image[..., 1] = np.clip(hsv_image[..., 1] * saturation_factor, 0, 255)
    return cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)

In [11]:
# Loop through each subfolder in the root folder
for subfolder in os.listdir(root_folder):
    subfolder_path = os.path.join(root_folder, subfolder)

    # Check if the subfolder is a directory
    if os.path.isdir(subfolder_path):
        # Loop through image files in the subfolder
        for filename in os.listdir(subfolder_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                image_path = os.path.join(subfolder_path, filename)

                # Load the image
                image = cv2.imread(image_path)

                # Apply random augmentations
                augmented_brightness = random_brightness(image)
                augmented_contrast = random_contrast(image)
                augmented_flip_lr = random_flip_left_right(image)
                augmented_flip_ud = random_flip_up_down(image)
                augmented_hue = random_hue(image)
                augmented_jpeg_quality = random_jpeg_quality(image)
                augmented_saturation = random_saturation(image)

                # Save the augmented images in the same subfolder
                output_folder = subfolder_path
                cv2.imwrite(os.path.join(output_folder, f'{filename}_augmented_brightness.jpg'), augmented_brightness)
                cv2.imwrite(os.path.join(output_folder, f'{filename}_augmented_contrast.jpg'), augmented_contrast)
                cv2.imwrite(os.path.join(output_folder, f'{filename}_augmented_flip_lr.jpg'), augmented_flip_lr)
                cv2.imwrite(os.path.join(output_folder, f'{filename}_augmented_flip_ud.jpg'), augmented_flip_ud)
                cv2.imwrite(os.path.join(output_folder, f'{filename}_augmented_hue.jpg'), augmented_hue)
                cv2.imwrite(os.path.join(output_folder, f'{filename}_augmented_jpeg_quality.jpg'), augmented_jpeg_quality)
                cv2.imwrite(os.path.join(output_folder, f'{filename}_augmented_saturation.jpg'), augmented_saturation)


In [12]:
# Loop through each subfolder in the root folder
for subfolder in os.listdir(root_folder):
    subfolder_path = os.path.join(root_folder, subfolder)

    # Check if the subfolder is a directory
    if os.path.isdir(subfolder_path):
        # Initialize a counter for image files
        image_count = 0

        # Loop through files in the subfolder
        for filename in os.listdir(subfolder_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                image_count += 1

        print(f"Subfolder '{subfolder}' contains {image_count} image(s).")

Subfolder 'LPPILOTSITTING2' contains 2984 image(s).
Subfolder 'LPPILOTLAT2' contains 2976 image(s).
Subfolder 'LPPILOTLAT1' contains 2280 image(s).
Subfolder 'AM12' contains 2984 image(s).


Duplicating the Labels

In [ ]:
# Define the path to the label Excel file
label_folder = '/content/drive/MyDrive/Label'
label_file = 'shortaxis_binary.xlsx'
xl = pd.ExcelFile(os.path.join(label_folder, label_file))

In [ ]:
# Define the path to the dataset folder
dataset_root = '/content/drive/MyDrive/dataset'

In [ ]:
# Create a dictionary to store DataFrames for different subfolders
subfolder_dfs = {}

In [32]:
# Create a dictionary to store DataFrames for different subfolders
subfolder_dfs = {}

# Iterate through subfolders in the dataset_root
for subfolder in os.listdir(dataset_root):
    subfolder_path = os.path.join(dataset_root, subfolder)

    if os.path.isdir(subfolder_path):
        # Extract the subfolder name and match it to the Excel tab
        tab_name = subfolder
        if tab_name in xl.sheet_names:
            # Read the corresponding Excel tab
            subfolder_df = xl.parse(tab_name)

            # Create a list to store modified rows
            modified_rows = []

            # Loop through the image files in the subfolder
            for filename in os.listdir(subfolder_path):
                if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                    # Extract the digits from the image file name (e.g., '206' from '206.jpg')
                    image_number = re.match(r'^\d+', filename).group()

                    # Find the corresponding row in the Excel tab based on the extracted digits
                    matching_rows = subfolder_df[subfolder_df['FILENAME'].str.contains(f'_{image_number}\.')]

                    if not matching_rows.empty:
                        # Update the 'FILENAME' column with the image name
                        matching_rows['FILENAME'] = filename
                        modified_rows.append(matching_rows)

            # Concatenate the modified rows and store them in the subfolder DataFrame
            if modified_rows:
                subfolder_result_df = pd.concat(modified_rows, ignore_index=True)
                subfolder_dfs[subfolder] = subfolder_result_df


<ipython-input-32-c87c2a1ded41>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_rows['FILENAME'] = filename
<ipython-input-32-c87c2a1ded41>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_rows['FILENAME'] = filename
<ipython-input-32-c87c2a1ded41>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [34]:
# Save the results for different subfolders in the label folder
for subfolder, subfolder_df in subfolder_dfs.items():
    output_file = os.path.join(label_folder, f'{subfolder}.xlsx')
    subfolder_df.to_excel(output_file, index=False)

print("Results saved as separate Excel files for different subfolders in the label folder.")

Results saved as separate Excel files for different subfolders in the label folder.
